In [44]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import f_regression
from sklearn.cross_validation import cross_val_score, cross_val_predict

In [ ]:
diabetes = datasets.load_diabetes()

In [ ]:
# View dict elements
diabetes.keys()

# View Data
print(diabetes.data.shape)
col_names = "age sex bmi map tc ldl hdl tch ltg glu".split()
data_frame = pd.DataFrame(diabetes.data, columns=col_names)
y = diabetes.target
sns.pairplot(data_frame)

(442, 10)


In [ ]:
# Check for extreme/suspicious values
for col_name in col_names:
    print(col_name,":",min(data_frame[col_name]),np.percentile(data_frame[col_name], 
        [25, 50, 75]),max(data_frame[col_name]))

In [ ]:
regr = linear_model.LinearRegression()

# LOO Cross-Validation 
scores = cross_val_score(regr, data_frame, y, cv=5)
print(scores)

First try LOO Cross-Validation using all possible models with 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 variables. For each run, calculate $$RMSE = \sqrt{\frac{1}{n}\sum \limits_{i=1}^n (y_i - \widehat{y}_{(-i)})^2}$$

Then try a forward selection algorithm and see what it picks. Use the AIC criterion (or BIC).

If time try LASSO for variable selection

In [ ]:
import itertools
from sklearn.model_selection import LeaveOneOut


def looRMSE(X, y):
    loo = LeaveOneOut()
    rmse = 0
    for train, test in loo.split(X):
        regr = linear_model.LinearRegression()
        regr.fit(X[train], y[train])
        rmse += (regr.predict(X[test])[0] - y[test][0])**2
        
    rmse = np.sqrt(rmse/(X.shape[0]))
    return rmse    
        
     
tested = []
result = []
for i in range(len(col_names)):
    for j in range(1, i + 1):
        testCombos = itertools.combinations(np.arange(i), j)
        for c in testCombos:
            if set(c) in tested:
                continue
            tested.append(set(c))
            testSet = data_frame.values[:, c]
            result.append(looRMSE(testSet, y))
            print("{}, {}".format(set(c), result[-1]))
result = np.array(result)
print("{:.2f}, {}".format(result.min(), tested[result.argmin()]))
        
    
    
        

In [ ]:
# Forward selection with AIC criterion
from sklearn.metrics import mean_squared_error
from scipy.stats import norm

class ForwardSelector:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.used = np.zeros((X.shape[1])) # variables used in model, start with none

    def calculate_aic(self, y, yhat, k):
        e = 1./(y.shape[0] - (k+1)) * np.sum((y - yhat)**2)
        
        g = norm.pdf(y, yhat, np.sqrt(e))
        N = np.sum(np.log(g))
        
        aic = 2*(k + 1) - 2*N
        return aic

    def forward_select(self):
        current_best = np.iinfo(np.uint64).max
        new_best = current_best - 1
        
        while new_best < current_best:
            current_best = new_best
            candidate_scores = np.empty((self.X.shape[1]))
            print(current_best)
            for i in range(self.X.shape[1]):
                if self.used[i] == 1:
                    continue
                candidate_model = np.zeros((self.X.shape[1]))
                candidate_model[self.used==1] = 1
                candidate_model[i] = 1
                dset = self.X[:, candidate_model==1]
                regr = linear_model.LinearRegression()
                regr.fit(dset, self.y)
                yhat = regr.predict(dset)
                candidate_scores[i] = self.calculate_aic(self.y, yhat, np.sum(candidate_model))
            new_best = np.min(candidate_scores)
            if new_best < current_best:
                self.used[np.argmin(candidate_scores)] = 1
                
                
fs = ForwardSelector(data_frame.values, y)
fs.forward_select()
print(fs.used)

        
        

In [ ]:
# View histogram of outcome variable
plt.hist(np.sqrt(y_train))

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(diabetes_X_train, diabetes_y_train)

In [ ]:
data_frame.shape[0]

In [ ]:
y